In [1]:
from confluent_kafka import Producer

In [2]:
class ExampleProducer:
    broker = "localhost:9092"
    producer = None

    def __init__(self, topic):
        self.producer = Producer({
            'bootstrap.servers': self.broker,
            'socket.timeout.ms': 100,
            'api.version.request': 'false',
            'broker.version.fallback': '0.9.0',
        }
        )
        self.topic = topic

    def delivery_report(self, err, msg):
        """ Called once for each message produced to indicate delivery result.
            Triggered by poll() or flush(). """
        if err is not None:
            print('Message delivery failed: {}'.format(err))
        else:
            print('Message delivered to {} [{}]'.format(
                msg.topic(), msg.partition()))

    def send_msg_async(self, msg):
        print("Send message asynchronously")
        self.producer.produce(
            self.topic,
            msg,
            callback=lambda err, original_msg=msg: self.delivery_report(err, original_msg
                                                                        ),
        )
        self.producer.flush()

    def send_msg_sync(self, msg):
        print("Send message synchronously")
        self.producer.produce(
            self.topic,
            msg,
            callback=lambda err, original_msg=msg: self.delivery_report(
                err, original_msg
            ),
        )
        self.producer.flush()



In [12]:
import pandas as pd
from io import BytesIO

In [23]:
from FlightRadar24.api import FlightRadar24API
import pandas as pd

# Get important data on current flights

def active_airplane(airline_icao):
    fr_api = FlightRadar24API()
    flight_list=[]
    flights = fr_api.get_flights(airline = airline_icao)
    for flight in flights :
        try :
            dest_iata = flight.destination_airport_iata
        except :
            dest_iata = None
        try :
            org_iata = flight.origin_airport_iata
        except :
            org_iata = None
        try :
            model = flight.aircraft_code
        except :
            model = None 
        try :
            registration = flight.registration
        except :
            registration = None
        try :
            speed = flight.ground_speed
        except :
            speed = None

        f = {
            "id" : flight.id,
            "dest_iata" : dest_iata,
            "org_iata" : org_iata,
            "model" : model, 
            "registration" : registration,
            "speed" : speed,
            "company" : airline_icao
            }
        flight_list.append(f)
    return flight_list

#Update active airplanes

def update_planes():
    df_airlines = pd.read_parquet("../data/Airlines.parquet")
    active_flights = []
    for airline in df_airlines["ICAO"]: # since the API only gave 1500 flights on one request I choose to request flights by airlines
        active_flights += active_airplane(airline)
    df_flights = pd.DataFrame(active_flights)
    df_flights = df_flights[df_flights["dest_iata"] != 'N/A'] #clean data
    df_flights = df_flights[df_flights["org_iata"] != 'N/A']

    df_flights.to_parquet("../data/Flights.parquet")
    return True

def planes():
    df_airlines = pd.read_parquet("../data/Airlines.parquet")
    active_flights = []
    for airline in df_airlines["ICAO"]: # since the API only gave 1500 flights on one request I choose to request flights by airlines
        active_flights += active_airplane(airline)
    df_flights = pd.DataFrame(active_flights)
    df_flights = df_flights[df_flights["dest_iata"] != 'N/A'] #clean data
    df_flights = df_flights[df_flights["org_iata"] != 'N/A']

    return df_flights.to_parquet()

In [24]:
#SENDING DATA TO KAFKA TOPIC
import json
from time import sleep, time, ctime


example_producer = ExampleProducer("test-topic")
# df = pd.read_parquet("../data/Airports.parquet")
# message = df.to_parquet()
# message = ["testing"+i for i in [str(k) for k in range(22)]]
# for mes in message :
#     example_producer.send_msg_async(mes)
#     sleep(1)

data = planes()
example_producer.send_msg_async(data)

Send message asynchronously
Message delivered to test-topic [0]
